In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

2023-01-13 12:18:22.946301: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/alexander.kell/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load eurosat dataset
dataset, info = tfds.load('eurosat/rgb', with_info=True)

2023-01-13 12:18:42.817326: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
train_ds = dataset['train']

In [4]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

normalized_ds = train_ds.map(lambda x: (normalization_layer(x['image']), x['label']))

# resize images to 224x224
IMG_SIZE = 224
def resize(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    return image, label

resized_ds = normalized_ds.map(resize)
resized_ds

# split into train and validation
train_ds = resized_ds.take(int(27000*0.6))
val_ds = resized_ds.skip(int(27000*0.2))
test_ds = resized_ds.skip(int(27000*0.6)).take(int(27000*0.2))

# batch and prefetch
BATCH_SIZE = 32
train_ds = train_ds.batch(BATCH_SIZE).prefetch(1)
val_ds = val_ds.batch(BATCH_SIZE).prefetch(1)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(1)




Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [5]:
# load resnet50
resnet50 = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))

# freeze layers
for layer in resnet50.layers:
    layer.trainable = False

# add new layers
x = tf.keras.layers.Flatten()(resnet50.output)

x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.1)(x)

x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.1)(x)

x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.1)(x)

x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dropout(0.1)(x)

x = tf.keras.layers.Dense(10, activation='softmax')(x)

model = tf.keras.Model(inputs=resnet50.input, outputs=x)

# compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# train model
history = model.fit(train_ds, epochs=10, validation_data=val_ds, verbose=1)


Epoch 1/10
  3/507 [..............................] - ETA: 27:31 - loss: 6.3170 - accuracy: 0.0729 

KeyboardInterrupt: 